In [1]:
#!/usr/bin/env python
# coding=utf-8
# -------- import basic package --------
import pdb
import math
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import warnings
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

import washer.utils.path.pathHandle as ph
import washer.utils.store as store
from washer.sample.feature import FeatSampler
from washer.repairer.isolatedRepairer import GeneralRepairer

from washer.demo.tianchi.mobile.system.utils.preProcess import *
from washer.demo.tianchi.mobile.system.utils.selectFeature import *

PATH_OFFLINE = "F:/codeGit/my project/python/dataset/tianchi/offline/"
PATH_ONLINE = "F:/codeGit/my project/python/dataset/tianchi/online/"
PATH_OF_DATAOUT = "F:/codeGit/my project/python/dataset/tianchi/submission/"
PATH_MODEL = "F:/codeGit/my project/python/dataset/tianchi/model/"

D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def getPolyFeature(df):
	print 'geting PolynomialFeature...'
	columns = df.columns
	df = getPolynomialFeature(df, columns)
	# print 'geting CrossFeature...'
	# df = getCrossFeature(df, columns, type = 'mutiple')
	# df = getCrossFeature(df, columns, type = 'sum')
	# df = getCrossFeature(df, columns, type = 'sub')
	# df = getCrossDiviFeature(df, columns)
	dr = GeneralRepairer()
	# print(df)
	pre_len = len(df.columns)
	ignore_cols =  ['item', 'label']
	for col in df.columns:
		if col not in ignore_cols:
			if (df[col].max() - df[col].min()) == 0:
				del df[col]
	print pre_len - len(df.columns)
	return df

In [3]:
def processTrainSet(df, isPolyFeatNeed = False):
	df = sampleBalance(df)
	if isPolyFeatNeed ==  True:
		df = getPolyFeature(df)
	df = preprocess(df, normlized = True)
	return df

In [4]:
def trainModel(X, y, model, saveMode = False):
	model = trainWithKfold(X, y, model, le)
	if saveMode == True:
		store.saveModelToPickle(PATH_MODEL + 'model_mnb_layer1.pkl', model)
	return model

In [25]:
df_train = pd.read_csv(PATH_OFFLINE + 'statisFeat_' + '2014-12-16_' + '4days.csv')

# df_train = getDatasetInItemTable(df_train)
# df_package = divideDataset(df_train, ratioList = [6, 4])
# df_train = df_package[0]
# df_cv = df_package[1]
# df_train = processTrainSet(df_train)

# df_train = sampleBalance(df_train)
# df = preprocess(df, normlized = True)
# model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, max_iter=2500, random_state = 40)
df_train = df_train[df_train.last_cart_h > 0]
df_train = df_train.drop(['user_id', 'item_id', 'item_category'], axis = 1)
print(df_train)
X, y, le = transform_dataset(df_train)
print(X[:10, :])

         uid_behav_cnt  uid_buy_cnt  user_buy_cnt  user_cnt  item_buy_cnt  \
187                  7          0.0           9.0       769           0.0   
348                  9          1.0           9.0       769           1.0   
386                  9          1.0           9.0       769           1.0   
423                  3          0.0           6.0       293           0.0   
443                  3          0.0           6.0       293           0.0   
446                  4          0.0           6.0       293           0.0   
456                  3          0.0           6.0       293           0.0   
460                  4          0.0           6.0       293           0.0   
481                  3          0.0           6.0       293           0.0   
518                  3          0.0           6.0       293           0.0   
603                  3          0.0           0.0        63           0.0   
619                  3          0.0           0.0        63           0.0   

In [26]:
## 随机森林
model = RandomForestClassifier(n_estimators=100, n_jobs=4, random_state=24)
model = trainModel(X, y, model, saveMode = False)

trainig model...
F1/P/R 15.24%/8.89%/53.33%

F1/P/R 9.01%/5.05%/41.67%

F1/P/R 17.46%/10.09%/64.71%

F1/P/R 8.33%/4.40%/80.00%

F1/P/R 14.95%/8.42%/66.67%

mean: F1/P/R 13.00%/7.37%/61.27%



In [28]:
## GBDT
model = GradientBoostingClassifier(max_depth=2)
model = trainModel(X, y, model, saveMode = False)

trainig model...
F1/P/R 2.08%/1.11%/16.67%

F1/P/R 0.00%/0.00%/0.00%

F1/P/R 0.00%/0.00%/0.00%

F1/P/R 0.00%/0.00%/0.00%

F1/P/R 4.08%/2.11%/66.67%

mean: F1/P/R 1.23%/0.64%/16.67%



In [29]:
## 测试数据集
df_test = pd.read_csv(PATH_OFFLINE + 'statisFeat_' + '2014-12-18_' + '4days.csv')
df_test = df_test[df_test.last_cart_h > 0]
df_test = df_test.drop(['user_id', 'item_id', 'item_category'], axis = 1)
# df['label'].apply(lambda x: int(x))
X_test, y_test, le = transform_dataset(df_test)
y_pred = model.predict(X_test)
calc_F1(y_test, y_pred)

F1/P/R 1.60%/0.82%/30.77%



(1.596806387225549, 0.819672131147541, 30.76923076923077)

In [ ]:
from sklearn import feature_selection as fsel
X_new = fsel.SelectKBest(fsel.chi2, k=8).fit_transform(X, y)
model = RandomForestClassifier(n_estimators=100, n_jobs=4, random_state=24)
trainModel(X_new, y, model, saveMode = False)

In [ ]:
## 提交
df_sub = pd.read_csv(PATH_OFFLINE + 'statisFeat_' + '2014-12-19_' + '4days.csv')
itemSet = pd.read_csv('F:/codeGit/dataset/tianchi_mobile/tianchi_fresh_comp_train_item.csv', usecols = ['item_id'])
itemSet = itemSet['item_id'].apply(lambda x: str(x))
df_sub = df_sub[df_sub['item_id'].isin(itemSet)]

itemList = df_sub['item_id']
X_sub = df_sub.drop(['user_id', 'item_id', 'item_category'], axis = 1).values
y_sub_pred = model.predict(X_sub)

predList = pd.DataFrame(y_sub_pred, index = itemList.index, columns = ['label'])
predList = predList[predList['label'] == 1]
predList[['user_id', 'item_id']].to_csv(PATH_OF_DATAOUT + 'tianchi_mobile_recommendation_predict.csv', mode = 'w', index = False)
